In [1]:

from mxnet import image
from mxnet import gluon
from mxnet import nd
from mxnet.gluon import nn,Block
from mxnet import init
from mxnet.gluon.model_zoo import vision as models
import mxnet as mx
import numpy as np
import random
import sys
sys.path.append('..')
import utils
from time import time
classes = ['background','p1','p2','p3','p4',
           'p5','p6','p7','p8','p9','p10',
           'p11','p12','p13','p14','p15',
           'p16','p17','p18','p19']
# RGB color for each class
colormap = [[0,0,0],[128,0,0],[0,128,0], [128,128,0], [0,0,128],
            [128,0,128],[0,64,128],[128,128,128],[64,0,0],[192,0,0],
            [64,128,0],[192,128,0],[64,0,128],[192,0,128],
            [64,128,128],[192,128,128],[0,64,0],[128,64,0],
            [0,192,0],[128,192,0]]

min_coordinate = [[504,697],[1063,606],[948,888],[286,902],[1079,1191],[1053,1507],[1048,1661],
                  [993,1729],[1025,1706],[415,1391],[1121,1355],[1138,1388],[1245,1294],
                  [1212,1477],[1170,1152],[1114,1756],[635,1090],[1081,1100],[350,1006]]
expand_size = 16

landmark_index = 9

# len(classes), len(colormap)



In [2]:
from mxnet import image
import sys

sys.path.append('..')
import utils
from time import time

data_root = '../data'
image_root = data_root + '/CephalometricLandmark/CroppedImage'
# image_root = data_root + '/CephalometricLandmark/ContrastImage1'
txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'
model_params_root = data_root + '/CephalometricLandmark/model_params'

def read_images(dataset_num=0):
    contrast_size = 0
    if dataset_num == 0:
        begin_index = 1
        end_index = 151
        contrast_size = 0
    elif dataset_num == 1:
        begin_index = 151
        end_index = 301
    else:
        begin_index = 301
        end_index = 401

    data, label = [None] * (end_index - begin_index) * (contrast_size + 1), [None] * (end_index - begin_index) * (
            contrast_size + 1)
    #     expand_label =  [None] * (end_index - begin_index)
    #     bounding_box = [[[2000,2000],[0,0],[0,0]] for i in range(19)]
    index = 0
    for contrast_index in range(contrast_size + 1):
#         image_root = data_root + '/CephalometricLandmark/ContrastImage' + str(contrast_index)
#         image_root = data_root + '/CephalometricLandmark/GaussianNoise' + str(contrast_index)
        for i in range(begin_index, end_index):
            image_filename = image_root + "/%02d/%03d.bmp" % (landmark_index + 1, i)
            txt_filename1 = txt_root + '/400_junior' + "/%03d.txt" % i
            txt_filename2 = txt_root + '/400_predict_average' + "/%03d.txt" % i
            # #         label_image[index] = nd.zeros_like(data[index])

            with open(txt_filename1, 'r') as f:
                txts = f.read().split()
            with open(txt_filename2, 'r') as f:
                txts1 = f.read().split()
#             x = int((int(txts[landmark_index].split(',')[0]) + int(txts1[landmark_index].split(',')[0]))/2) - min_coordinate[landmark_index][0]
#             y = int((int(txts[landmark_index].split(',')[1]) + int(txts1[landmark_index].split(',')[1]))/2) - min_coordinate[landmark_index][1]
            #         label_image[index][y-expand_size:y+expand_size,x-expand_size:x+expand_size] = colormap[landmark_index+1]
            rand_numx = 0
            rand_numy = 0
            random_range = 20
            if dataset_num == 0:
                rand_numx = random.randint(-random_range, random_range)
                rand_numy = random.randint(-random_range, random_range)
            
            
            crop_x = int(txts1[landmark_index].split(',')[0]) - min_coordinate[landmark_index][0] + rand_numx
            crop_y = int(txts1[landmark_index].split(',')[1]) - min_coordinate[landmark_index][1] + rand_numy
            crop_minx = crop_x - 32
            if crop_minx < 0:
                crop_minx = 0
            if crop_minx + 64 > 640:
                crop_minx = 640 - 64
            
            crop_miny = crop_y - 32
            if crop_miny < 0:
                crop_miny = 0
            if crop_miny + 64 > 640:
                crop_miny = 640 - 64
            
            x = int(txts[landmark_index].split(',')[0]) - min_coordinate[landmark_index][0] - crop_minx
            y = int(txts[landmark_index].split(',')[1]) - min_coordinate[landmark_index][1] - crop_miny
            print(x,y)
                
            minx = x - expand_size
            maxx = x + expand_size
            if minx < 0:
                minx = 0
            if minx >= 64:
                minx = 63
            if maxx < 0:
                maxx = 0
            if maxx >= 64:
                maxx = 63

            miny = y - expand_size
            maxy = y + expand_size
            if miny < 0:
                miny = 0
            if miny >= 64:
                miny = 63
            if maxy < 0:
                maxy = 0
            if maxy >= 64:
                maxy = 63
            data[index] = image.imread(image_filename)[crop_miny:crop_miny+64,crop_minx:crop_minx+64]
            label[index] = nd.zeros((data[index].shape[0], data[index].shape[1]))
#             print(label[index].shape)
            print(miny,maxy,minx,maxx)
            if miny != maxy and minx != maxx:
                label[index][miny:maxy, minx:maxx] = 1
#             label[index] = nd.flip(label[index],0)
#             print(data[index].shape,label[index].shape)
            index += 1
            
    return data, label

# train_images, train_label_images, train_labels,bounding_boxes= read_images(0)


In [3]:
# imgs = [train_images[0],train_label_images[0]]
# # print(bounding_boxes)
# # print(type(train_images))
# landmarkindex = 2
# margin_size = 100
# for i in range(19):
# #     imgs += [train_images[i][900:1100,800:1000],train_labels[i][990:1050,800:860]]
# #     x = train_labels[i][1][0]
# #     y = train_labels[i][1][1]
# #     size = 40
# #     imgs += [train_images[i][y-size:y+size,x-size:x+size],train_label_images[i][y-size:y+size,x-size:x+size]]
    
#     minx = bounding_boxes[0][0]
#     miny = bounding_boxes[0][1]
#     maxx = bounding_boxes[1][0]
#     maxy = bounding_boxes[1][1]

#     imgs += [train_images[i][miny-margin_size:maxy+margin_size,minx-margin_size:maxx+margin_size],train_label_images[i][miny-margin_size:maxy+margin_size,minx-margin_size:maxx+margin_size]]
# # print(train_labels[0][981:1001,819:839])
# utils.show_images(imgs, nrows=19, ncols=2,figsize=(12,160))
# [im.shape for im in imgs]

In [4]:

# imgs = []
# landmark_index = 0
# for i in range(10):
#     crop_image = image_crop(train_images[i],bounding_boxes[0],bounding_boxes[1])
#     crop_label = image_crop(train_label_images[i],bounding_boxes[0],bounding_boxes[1])
#     imgs += [crop_image,crop_label]
# utils.show_images(imgs, nrows=10, ncols=2,figsize=(12,160))

In [5]:
from mxnet import gluon
from mxnet import nd
import random

rgb_mean = nd.array([0.485, 0.456, 0.406])
rgb_std = nd.array([0.229, 0.224, 0.225])

def normalize_image(data):
#     mx.random.seed(np.random.randint(1, 1000))
#     noise = mx.nd.random.normal(0, 1, shape=(640, 640, 1), dtype=np.float32)
#     return ((data.astype('float32')+ 10*noise).clip(0, 255) / 255 - rgb_mean) / rgb_std
    return (data.astype('float32') / 255 - rgb_mean) / rgb_std

class VOCSegDataset(gluon.data.Dataset):

    def __init__(self, dataset_num, crop_size):
        self.crop_size = crop_size
        self.data, self.label = read_images(dataset_num=dataset_num)
        self.data[:] = [normalize_image(im) for im in self.data]
#         for i in range(len(self.data)):
#             tmp = image_crop(self.data[i], self.bounding_box[0], self.bounding_box[1],*self.crop_size)
#             self.data[i].reshape(tmp.shape)
#             self.data[i] = tmp.transpose((2,0,1))
#             tmp = image_crop(self.label_image[i], self.bounding_box[0], self.bounding_box[1],*self.crop_size)
#             self.label_image[i].reshape(tmp.shape)
#             self.label_image[i] = tmp.transpose((2,0,1))
#         print('Read '+str(len(self.data))+' examples')

    def __getitem__(self, idx):
#         data,label = image_crop(self.data[idx],self.label[idx],self.bounding_box[0], self.bounding_box[1],*self.crop_size)
        data = self.data[idx]
        label = self.label[idx]
        
#         aug1 = image.HorizontalFlipAug(1)
# #         aug2 = image.BrightnessJitterAug(1)
# #         aug3 = image.ContrastJitterAug(1)
#         if random.random() > 0.5:
#         print(data.shape)
#         data =  nd.flip(data, axis=0)
#         label = nd.flip(label,0)
#         data = aug2(data)
#         data = aug3(data)
            
        return data.transpose((2,0,1)), label

    def __len__(self):
        return len(self.data)

In [6]:
input_shape = (64, 64)
voc_train = VOCSegDataset(0, input_shape)
voc_test1 = VOCSegDataset(1, input_shape)
# voc_test2 = VOCSegDataset(2, input_shape)

47 50
34 63 31 63
28 40
24 56 12 44
50 42
26 58 34 63
39 42
26 58 23 55
38 30
14 46 22 54
20 25
9 41 4 36
48 25
9 41 32 63
13 16
0 32 0 29
43 26
10 42 27 59
27 22
6 38 11 43
23 51
35 63 7 39
36 34
18 50 20 52
21 15
0 31 5 37
26 52
36 63 10 42
25 36
20 52 9 41
25 24
8 40 9 41
18 34
18 50 2 34
45 12
0 28 29 61
24 42
26 58 8 40
50 36
20 52 34 63
31 25
9 41 15 47
31 32
16 48 15 47
39 29
13 45 23 55
37 51
35 63 21 53
16 52
36 63 0 32
33 28
12 44 17 49
17 12
0 28 1 33
29 36
20 52 13 45
25 31
15 47 9 41
20 51
35 63 4 36
30 23
7 39 14 46
18 21
5 37 2 34
48 48
32 63 32 63
25 32
16 48 9 41
46 16
0 32 30 62
49 23
7 39 33 63
31 19
3 35 15 47
15 28
12 44 0 31
39 25
9 41 23 55
37 45
29 61 21 53
13 17
1 33 0 29
16 44
28 60 0 32
19 29
13 45 3 35
29 20
4 36 13 45
43 21
5 37 27 59
49 44
28 60 33 63
23 20
4 36 7 39
28 43
27 59 12 44
34 50
34 63 18 50
50 44
28 60 34 63
20 52
36 63 4 36
38 12
0 28 22 54
25 43
27 59 9 41
51 13
0 29 35 63
38 24
8 40 22 54
41 28
12 44 25 57
17 46
30 62 1 33
29 44
28 60 13 45


In [7]:
batch_size = 8
train_data = gluon.data.DataLoader(
    voc_train, batch_size, shuffle=True,last_batch='discard')
test_data = gluon.data.DataLoader(
    voc_test1, batch_size,last_batch='discard')

for data, label in train_data:
    print(data.shape)
    print(label.shape)
    break

(8, 3, 64, 64)
(8, 64, 64)


In [8]:
from mxnet.gluon import nn
from mxnet import nd

conv = nn.Conv2D(10, kernel_size=4, padding=1, strides=2)
conv_trans = nn.Conv2DTranspose(3, kernel_size=4, padding=1, strides=2)

conv.initialize()
conv_trans.initialize()

x = nd.random.uniform(shape=(1,3,64,64))
y = conv(x)
print('Input:', x.shape)
print('After conv:', y.shape)
print('After transposed conv', conv_trans(y).shape)

Input: (1, 3, 64, 64)
After conv: (1, 10, 32, 32)
After transposed conv (1, 3, 64, 64)


In [9]:
from mxnet.gluon.model_zoo import vision as models
pretrained_net = models.densenet121(pretrained=True)

# (pretrained_net.features[-4:], pretrained_net.output)

In [10]:
net = nn.HybridSequential()
for layer in pretrained_net.features[:-2]:
    net.add(layer)

x = nd.random.uniform(shape=(1,3,*input_shape))
print('Input:', x.shape)
print('Output:', net(x).shape)

Input: (1, 3, 64, 64)
Output: (1, 1024, 2, 2)


In [11]:
num_classes = len(classes)

with net.name_scope():
    net.add(
        nn.Conv2D(2, kernel_size=1),
        nn.Conv2DTranspose(2, kernel_size=64, padding=16,strides=32)
    )
print(net)

HybridSequential(
  (0): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
  (2): Activation(relu)
  (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
  (4): HybridSequential(
    (0): HybridConcurrent(
      (0): Identity(
      
      )
      (1): HybridSequential(
        (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (1): Activation(relu)
        (2): Conv2D(64 -> 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=128)
        (4): Activation(relu)
        (5): Conv2D(128 -> 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (1): HybridConcurrent(
      (0): Identity(
      
      )
      (1): HybridSequential(
     

In [12]:
def bilinear_kernel(in_channels, out_channels, kernel_size):
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)
    weight = np.zeros(
        (in_channels, out_channels, kernel_size, kernel_size),
        dtype='float32')
    weight[range(in_channels), range(out_channels), :, :] = filt
    return nd.array(weight)

In [13]:
# from matplotlib import pyplot as plt

# x = train_images[0]
# print('Input', x.shape)
# x = x.astype('float32').transpose((2,0,1)).expand_dims(axis=0)/255

# conv_trans = nn.Conv2DTranspose(
#     3, in_channels=3, kernel_size=8, padding=2, strides=4)
# conv_trans.initialize()
# conv_trans(x)
# conv_trans.weight.set_data(bilinear_kernel(3, 3, 8))


# y = conv_trans(x)
# y = y[0].clip(0,1).transpose((1,2,0))
# print('Output', y.shape)

# plt.imshow(y.asnumpy())
# plt.show()

In [14]:
from mxnet import init

conv_trans = net[-1]
conv_trans.initialize(init=init.Zero())
net[-2].initialize(init=init.Xavier())

x = nd.zeros((batch_size, 3, *input_shape))
net(x)

shape = conv_trans.weight.data().shape
conv_trans.weight.set_data(bilinear_kernel(*shape[0:3]))

In [15]:
import sys
sys.path.append('..')
import utils

loss = gluon.loss.SoftmaxCrossEntropyLoss(axis=1)
# loss = gluon.loss.SigmoidBinaryCrossEntropyLoss()

ctx = utils.try_all_gpus()
net.collect_params().reset_ctx(ctx)

trainer = gluon.Trainer(net.collect_params(),
                        'SGD', {'learning_rate': 0.02, 'wd':1e-3})

utils.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=100)


In [16]:
# net.save_params(model_params_root + "/net2-mark10-resnet18-epochs100.json")

In [17]:
import mxnet as mx
def predict(im):
#     mx.random.seed(np.random.randint(1, 1000))
#     noise = mx.nd.random.normal(0, 1, shape=(640, 640, 1), dtype=np.float32)
#     data = ((im.astype('float32')+ 5*noise).clip(0, 255) / 255 - rgb_mean) / rgb_std
    data = normalize_image(im)
#     data = (im.astype('float32') / 255 - rgb_mean) / rgb_std
    data = data.transpose((2,0,1)).expand_dims(axis=0)
    yhat = net(data.as_in_context(ctx[0]))
    pred = nd.argmax(yhat, axis=1)
    return pred.reshape((pred.shape[1], pred.shape[2]))

def label2image(pred):
    x = pred.astype('int32').asnumpy()
    cm = nd.array(colormap).astype('uint8')
    return nd.array(cm[x,:])

In [18]:
test_images,test_labels = read_images(1)

30 29
13 45 14 46
5 25
9 41 0 21
2 10
0 26 0 18
17 6
0 22 1 33
36 51
35 63 20 52
-13 -10
0 6 0 3
22 19
3 35 6 38
9 17
1 33 0 25
18 23
7 39 2 34
4 1
0 17 0 20
29 37
21 53 13 45
28 35
19 51 12 44
33 38
22 54 17 49
25 31
15 47 9 41
31 32
16 48 15 47
22 26
10 42 6 38
39 44
28 60 23 55
59 68
52 63 43 63
25 36
20 52 9 41
34 24
8 40 18 50
54 60
44 63 38 63
34 22
6 38 18 50
47 34
18 50 31 63
60 72
56 63 44 63
29 39
23 55 13 45
41 35
19 51 25 57
25 25
9 41 9 41
40 29
13 45 24 56
46 40
24 56 30 62
25 15
0 31 9 41
34 23
7 39 18 50
92 69
53 63 63 63
25 30
14 46 9 41
19 17
1 33 3 35
40 44
28 60 24 56
30 45
29 61 14 46
41 51
35 63 25 57
34 31
15 47 18 50
38 46
30 62 22 54
32 39
23 55 16 48
63 0
0 16 47 63
27 42
26 58 11 43
43 25
9 41 27 59
39 -43
0 0 23 55
23 17
1 33 7 39
46 33
17 49 30 62
45 39
23 55 29 61
42 50
34 63 26 58
2 27
11 43 0 18
18 31
15 47 2 34
31 27
11 43 15 47
24 23
7 39 8 40
33 53
37 63 17 49
18 30
14 46 2 34
21 -8
0 8 5 37
11 9
0 25 0 27
25 41
25 57 9 41
83 -21
0 0 63 63
32 17
1 33 

In [19]:
def evaluate_acc(result, label, index):
    if(len(np.where(result.asnumpy()>0)[1]) ==0 or len(np.where(label.asnumpy()>0)[1]) ==0):
        return False,False,False,False
#     result_maxx = np.max(np.where(result.asnumpy()>0)[1])
#     result_minx = np.min(np.where(result.asnumpy()>0)[1])
#     result_maxy = np.max(np.where(result.asnumpy()>0)[0])
#     result_miny = np.min(np.where(result.asnumpy()>0)[0])

#     result_centerx = int((result_maxx + result_minx)/2)
#     result_centery = int((result_maxy + result_miny)/2)
    txt_filename1 = txt_root + '/400_junior' + "/%03d.txt" % index
    txt_filename2 = txt_root + '/400_predict_average' + "/%03d.txt" % index

    with open(txt_filename1, 'r') as f:
        txts = f.read().split()
    with open(txt_filename2, 'r') as f:
        txts1 = f.read().split()
    x = int(txts[landmark_index].split(',')[0])- int(txts1[landmark_index].split(',')[0]) + 32
    y = int(txts[landmark_index].split(',')[1])- int(txts1[landmark_index].split(',')[1]) + 32
    
    result_avgx = np.average(np.where(result.asnumpy()>0)[1])
    result_avgy = np.average(np.where(result.asnumpy()>0)[0])

#     label_maxx = np.max(np.where(label.asnumpy()>0)[1])
#     label_minx = np.min(np.where(label.asnumpy()>0)[1])
#     label_maxy = np.max(np.where(label.asnumpy()>0)[0])
#     label_miny = np.min(np.where(label.asnumpy()>0)[0])
    
#     label_centerx = int((label_maxx + label_minx)/2)
#     label_centery = int((label_maxy + label_miny)/2)
    
#     lable_avgx = np.average(np.where(label.asnumpy()>0)[1])
#     lable_avgy = np.average(np.where(label.asnumpy()>0)[0])
    
#     d1 = pow((result_centerx - label_centerx),2) + pow((result_centery - label_centery),2)
#     print(result_avgx,result_avgy)
    d2 = pow((result_avgx - x),2) + pow((result_avgy - y),2)
#     print("============================")
#     print(np.where(result.asnumpy()>0))
#     print(result_centerx,label_centerx)
#     print(result_centery,label_centery)
#     print(d1,d2)
    return (d2 < 400, d2 < 625, d2 < 900, d2 < 1600)

In [20]:
n = len(test_images)
# n = 10
imgs = []
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0

# print(test_images[0])
for i in range(n):
    x = test_images[i]
#     print(x.shape)
#     start = time()
    result = predict(x)
    f1,f2,f3,f4 = evaluate_acc(result,test_labels[i],i+151)
    acc1 += int(f1)
    acc2 += int(f2)
    acc3 += int(f3)
    acc4 += int(f4)


#     print(type(np.where(result.asnumpy()>0)[0]))
#     print(np.where(test_labels[i].asnumpy()>0))
    pred = label2image(predict(x))
    imgs += [x, pred, label2image(test_labels[i])]
print(acc1/n,"  ", acc2/n,"  ", acc3/n,"  ", acc4/n)
# utils.show_images(imgs, nrows=n, ncols=3, figsize=(6,20))


0.2866666666666667    0.36    0.44666666666666666    0.5666666666666667


In [21]:
# test_images,test_labels = read_images(18,2)

In [22]:
# n = 10
# imgs = []
# for i in range(n):
#     x = test_images[i]
#     imgs += [x, label2image(test_labels[i]), label2image(test_labels[i])]
# # 
# utils.show_images(imgs, nrows=n, ncols=3, figsize=(6,10))


In [23]:
for i in range(0):
    print(i)